In [2]:
import re
from collections import Counter

In [6]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    "return (known([word]) or known(reductions(word)) or [word])"
    return (known(doubling1(word)) or known(doubling2(word)) or known(reductions(word)) or known(edits1(word)) or  known(edits2(word)) or [word] or known([word]))

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def vowel1(word):
    vowels = 'aeiou'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in vowels]
    return(replaces)

def doubling1(word):
    "single letter -> doubling letters (ex. aple -> apple)"
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    inserts    = [L + R[0] + R               for L, R in splits if R]
    return set(inserts)

def doubling2(word):
    "single letter -> doubling letters"
    return (d2 for d1 in doubling1(word) for d2 in doubling1(d1))

def reductions(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word))]
    deletes    = [L + R[1:]               for L, R in splits if R and L and L[-1] == R[0]]
    return set(deletes)

In [4]:
def unit_tests():
    assert correction('speling') == 'spelling'              # insert
    assert correction('korrectud') == 'corrected'           # replace 2
    assert correction('bycycle') == 'bicycle'               # replace
    assert correction('inconvient') == 'inconvenient'       # insert 2
    assert correction('arrainged') == 'arranged'            # delete
    assert correction('peotry') =='poetry'                  # transpose
    assert correction('peotryy') =='poetry'                 # transpose + delete
    assert correction('word') == 'word'                     # known
    assert correction('quintessential') == 'quintessential' # unknown
    assert words('This is a TEST.') == ['this', 'is', 'a', 'test']
    assert Counter(words('This is a test. 123; A TEST this is.')) == (
           Counter({'123': 1, 'a': 2, 'is': 2, 'test': 2, 'this': 2}))
    assert len(WORDS) == 32192
    assert sum(WORDS.values()) == 1115504
    assert WORDS.most_common(10) == [
     ('the', 79808),
     ('of', 40024),
     ('and', 38311),
     ('to', 28765),
     ('in', 22020),
     ('a', 21124),
     ('that', 12512),
     ('he', 12401),
     ('was', 11410),
     ('it', 10681)]
    assert WORDS['the'] == 79808
    assert P('quintessential') == 0
    assert 0.07 < P('the') < 0.08
    return 'unit_tests pass'

def spelltest(tests, verbose=True):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.perf_counter()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.perf_counter() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

In [7]:
spelltest(Testset(open('spell-testset2.txt')))

correction(emploies) => employed (156); expected employees (37)
correction(hierachial) => hierachial (0); expected hierarchal (0)
correction(adabtable) => adabtable (0); expected adaptable (0)
correction(latter) => matter (365); expected later (334)
correction(graphicaly) => graphicaly (0); expected graphically (0)
correction(academicly) => academic (4); expected academically (0)
correction(thermawere) => thermawere (0); expected thermawear (0)
correction(thermawhere) => thermawhere (0); expected thermawear (0)
correction(plesent) => present (328); expected pleasant (96)
correction(wagted) => wanted (213); expected weighted (2)
correction(orentated) => orentated (0); expected orientated (0)
correction(inut) => nut (10); expected input (1)
correction(pivting) => pitting (4); expected pivoting (0)
correction(manualy) => manual (17); expected manually (0)
correction(contuned) => contused (15); expected continued (291)
correction(unresloved) => unresloved (0); expected unresolved (0)
corre